In [21]:
import geopandas as gpd
import pandas as pd
import sys
import numpy as np
import rasterio as rs
from rasterio.mask import mask

sys.path.append('../../src/')
import utils.quick_viz as make
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
PIXEL_SIZE = 10
PIXEL_AREA_HA = (PIXEL_SIZE / 100) * (PIXEL_SIZE / 100)
PIXEL_AREA_HA

0.010000000000000002

In [34]:
prj_districts = gpd.read_file('../../data/epa_districts/Project_Districts.shp')
land_use_map = 'final_2024-10-29.tif'
zonal_stats = []
lu_cats = {
    0: "no_tree",
    1: "monoculture",
    2: "agroforestry",
    3: "natural"
}
with rs.open(land_use_map) as src:
    
    for _, district in prj_districts.iterrows():

        # Create mask, flatten and remove no data vals
        out_image, out_transform = mask(src, [district.geometry], crop=True)
        out_image = out_image[0] 
        valid_pixels = out_image[out_image != src.nodata]

        # Calculate the pixel count for each unique value (land use class)
        unique, counts = np.unique(valid_pixels, return_counts=True)
        land_use_stats = dict(zip(unique, counts))
      
        # Convert pixel counts to areas in hectares
        # Append the results for the current district
        row = {"district": district.ADM2_EN}
        for lu_class, area in land_use_stats.items():
            land_use_stats[lu_class] *= PIXEL_AREA_HA
            land_use_name = lu_cats.get(int(lu_class), f"unknown_{lu_class}")
            row[land_use_name] = area
        zonal_stats.append(row)

zonal_stats_df = pd.DataFrame(zonal_stats)

In [32]:
zonal_stats_df

,district,no_tree,monoculture,agroforestry,natural
0,Adansi South,5795.65,1043.97,52383.14,16194.22
1,Asante Akim South,4490.38,637.21,94889.69,14399.55
2,Assin North,3275.58,710.26,65038.86,2885.00
3,Atwima Mponua,11152.65,4991.58,155096.26,15562.04
4,Bawku West,88494.17,4.91,16001.45,2293.40
5,Bosome Freho,3444.50,723.70,40600.35,11620.22
6,Builsa South,96041.02,0.89,24850.17,3572.99
7,Daffiama Bussie Issa,110022.25,0.37,17223.58,18600.70
8,Juaben Municipal,1339.32,535.35,12058.22,3050.61
9,Kasena Nankana West,65773.44,12.19,19392.95,2429.92
